# **IMDB Dataset of 50K Movie Reviews**

In [5]:
!pip install kaggle

## **Importing the Dependenciess**

In [6]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Data Collection through kaggle API**

In [7]:
kaggle_dictionary=json.load(open('kaggle.json'))

In [8]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [9]:
# Setup kaggle credendial as environment variables

os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 39.2MB/s]
100% 25.7M/25.7M [00:00<00:00, 133MB/s] 


In [11]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [12]:
#unzipthe dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [13]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the Data set**

In [15]:
df=pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
df.shape

(50000, 2)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [18]:
df['sentiment'].value_counts() #To if our data is balanced or imbalanced

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [22]:
#now we will replace positive =1 and negative=0 in the column sentiment
df.replace({'positive':1,'negative':0},inplace=True)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [23]:
df['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [26]:
#split data into training data and test data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [27]:
print(train_df.shape)
print(test_df.shape)

(40000, 2)
(10000, 2)


#**Data preprocessing**

In [29]:
#Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_df['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_df['review']),maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_df['review']),maxlen=200)

In [30]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [31]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [33]:
y_train = train_df['sentiment']
y_test = test_df['sentiment']

In [34]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [36]:
# Build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(x_train, y_train, epochs=5,batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 236s 457ms/step - loss: 0.4193 - accuracy: 0.8036 - val_loss: 0.3560 - val_accuracy: 0.8614
Epoch 2/5
500/500 [==============================] - 200s 401ms/step - loss: 0.2697 - accuracy: 0.8923 - val_loss: 0.3444 - val_accuracy: 0.8656
Epoch 3/5
500/500 [==============================] - 202s 403ms/step - loss: 0.2239 - accuracy: 0.9122 - val_loss: 0.3021 - val_accuracy: 0.8749
Epoch 4/5
500/500 [==============================] - 201s 402ms/step - loss: 0.1835 - accuracy: 0.9306 - val_loss: 0.3093 - val_accuracy: 0.8735
Epoch 5/5
500/500 [==============================] - 200s 400ms/step - loss: 0.1552 - accuracy: 0.9423 - val_loss: 0.3255 - val_accuracy: 0.8749


In [40]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss:{loss}')
print(f'Test Accuracy:{accuracy}')

313/313 [==============================] - 20s 62ms/step - loss: 0.3146 - accuracy: 0.8756
Test Loss:0.31456804275512695
Test Accuracy:0.8755999803543091


## **Building a predictive system**

In [41]:
def predict_sentiment(review):
  # Tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
  return sentiment

In [42]:
# Example usage
new_review2 ='This movie was fantastic. I loved it.'
sentiment = predict_sentiment(new_review2)
print('The sentiment of the review is:',sentiment)

1/1 [==============================] - 0s 294ms/step
The sentiment of the review is: positive


In [43]:
# Example usage
new_review1 ='This movie was not that good.'
sentiment = predict_sentiment(new_review1)
print('The sentiment of the review is:',sentiment)

1/1 [==============================] - 0s 108ms/step
The sentiment of the review is: negative
